<a href="https://colab.research.google.com/github/afif-af/ml_pytorch_scikitlearn/blob/main/Chapter08ml_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files
# files.upload()

Saving aclImdb_v1.tar.gz to aclImdb_v1.tar.gz
Buffered data was truncated after reaching the output size limit.

In [ ]:
import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz')as tar:
  tar.extractall()

In [ ]:
import pyprind
import pandas as pd
import os
import sys
basepath='aclImdb'
labels={'pos':1,'neg':0}
pbar=pyprind.ProgBar(50000, stream=sys.stdout)
df=pd.DataFrame()
for s in ('test', 'train'):
  for l in ('pos', 'neg'):
    path =os.path.join(basepath, s, l)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
        text=infile.read()
      df=df.append([[text, labels[l]]],
                   ignore_index=True)
      pbar.update()
df.columns=['review','sentiment']


In [ ]:
import numpy as np
np.random.seed(0)
df= df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')


In [16]:
df=pd.read_csv('movie_data.csv', encoding='utf=-8')
df=df.rename(columns={"0": "review", "1": "sentiment"})
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [17]:
df.shape

(50000, 2)

In [18]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                'The sun is shining, the weather is sweet,'
                'and one and one is two'])
bag = count.fit_transform(docs)


In [19]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [20]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf=TfidfTransformer(use_idf=True,
                       norm='l2',
                       smooth_idf=True)

np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [23]:
df.loc[0, 'review'][-50:]


'is seven.<br /><br />Title (Brazil): Not Available'

In [25]:
import re
def preprocessor(text):
  text=re.sub('<[^>]*>',' ', text)
  emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                       text)
  text =(re.sub('[\W]+',' ',text.lower()) + ' '.join(emoticons).replace('-',''))
  return text

In [26]:
preprocessor ("</a>This :) is :( a test :-)!")


' this is a test :) :( :)'

In [27]:
df['review']=df['review'].apply(preprocessor)

In [31]:
def tokenizer(text):
  return text.split()

In [32]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [35]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]


In [37]:
tokenizer_porter('runners line running and thus they run')

['runner', 'line', 'run', 'and', 'thu', 'they', 'run']

In [39]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
[w for w in tokenizer_porter('a runner likes'
' running and runs a lot')
if w not in stop]


['runner', 'like', 'run', 'run', 'lot']

In [45]:
X_train=df.loc[:25000, 'review'].values
y_train=df.loc[:25000, 'sentiment'].values
x_test=df.loc[25000:, 'review'].values
y_test=df.loc[25000:, 'sentiment'].values

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(strip_accents=None,
                      lowercase=False,
                      preprocessor=None)
small_param_grid=[
    {
        'vect__ngram_range':[(1,1)],
        'vect__stop_words':[None],
        'vect__tokenizer':[tokenizer, tokenizer_porter],
        'clf__penalty':['l2'],
        'clf__C':[1.0, 10.0]
    },
    {
        'vect__ngram_range':[(1,1)],
        'vect__stop_words':[stop,None],
        'vect__tokenizer':[tokenizer, tokenizer_porter],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty':['l2'],
        'clf__C':[1.0,10.0]
    }
]

lr_tfidf=Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])

gs_lr_tfidf=GridSearchCV(lr_tfidf, small_param_grid,
                         scoring='accuracy', cv=5,
                         n_jobs=1,
                         verbose=2)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x785d3c0bd120>; total time=   5.5s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x785d3c0bd120>; total time=   4.5s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x785d3c0bd120>; total time=   6.2s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x785d3c0bd120>; total time=   4.4s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x785d3c0bd120>; total time=   4.5s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x785d3c0bd120>,
                                              <function tokenizer_porter at 0x785d3ae06340>...
                                                'again', 'against', 'ain',
                                                'all', 'am', 'an', 'and', 'any',
                                                'are', 'aren', "aren't", 'as',
                                                'at', 'be', 'because', 'been',
                                                'before', 'being', 'below',
                                                'between', 'both', 'but', 'by',
                                                'can', 'couldn', "couldn't", ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x785d3c0bd120>,
                                              <function tokenizer_porter at 0x785d3ae06340>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [49]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_}')
clf=gs_lr_tfidf.best_estimator_

X_test_full = df.loc[25000:, 'review'].values
y_test_full = df.loc[25000:, 'sentiment'].values

print(f'Test Accuracy: {clf.score(X_test_full, y_test_full)}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x785d3c0bd120>}
CV Accuracy: 0.8972842471505699
Test Accuracy: 0.89856


In [52]:
import numpy as np
import re
from nltk.corpus import stopwords
stop=stopwords.words('english')
def tokenizer(text):
  text=re.sub('<[^>]*>', '',text)
  emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                       text.lower())
  text=re.sub('[\W]+',' ',text.lower()) \
                + ' '.join(emoticons).replace('-','')
  tokenized=[w for w in text.split() if w not in stop]
  return tokenized

In [56]:
def stream_docs(path):
  with open(path, 'r', encoding='utf-8') as csv:
    next(csv)
    for line in csv:
      line = line.strip().split(',')
      text = ','.join(line[:-1])
      label = int(line[-1])
      yield text, label

In [57]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [67]:
def get_minibatch(doc_stream, size):
  docs, y=[], []
  try:
    for _ in range(size):
      text, label=next(doc_stream)
      docs.append(text)  # Append the text review
      y.append(label)    # Append the label
  except StopIteration:
    return None, None
  return docs, y

In [70]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer
                         )
clf = SGDClassifier(loss='log_loss', random_state=1)
doc_stram=stream_docs(path='movie_data.csv')

In [71]:
import pyprind
pbar=pyprind.ProgBar(45)
classes=np.array([0,1])
for _ in range(45):
  X_train, y_train, =get_minibatch(doc_stram, size=1000)
  if not X_train:
    break
  X_train =vect.transform(X_train)
  clf.partial_fit(X_train, y_train,classes=classes)
  pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:40


In [72]:
X_test, y_test=get_minibatch(doc_stram, size=5000)
X_test=vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [73]:
clf = clf.partial_fit(X_test, y_test)

In [74]:
import pandas as pd
df=pd.read_csv('movie_data.csv', encoding='utf-8')
df=df.rename(columns={"0":"review","1":"sentiment"})

In [76]:
from sklearn.feature_extraction.text import CountVectorizer
count=CountVectorizer(stop_words='english',
                      max_df=.1,
                      max_features=5000)
X=count.fit_transform(df['review'].values)

In [77]:
from sklearn.decomposition import LatentDirichletAllocation
lda=LatentDirichletAllocation(n_components=10,
                              random_state=123,
                              learning_method='batch')
X_topics=lda.fit_transform(X)

In [78]:
lda.components_.shape

(10, 5000)

In [79]:
n_top_words=5
feature_names=count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1) }:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                    [:-n_top_words -1:-1]]))


Topic 1:
horror worst script effects budget
Topic 2:
dvd watched video music guy
Topic 3:
war american series history documentary
Topic 4:
game killer murder thriller crime
Topic 5:
kids comedy episode series school
Topic 6:
family woman mother beautiful feel
Topic 7:
role performance comedy john plays
Topic 8:
action horror john effects dr
Topic 9:
book version original read music
Topic 10:
action wife father police james


In [80]:
horror =X_topics[:,5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
  print(f'\n Horror moview #{(iter_idx +1)}:')
  print(df['review'][movie_idx][:300], '...')



 Horror moview #1:
"Come Undone" appears to elicit a lot of opinions among the contributors to this forum. Granted, it's a film that promises a take on gay life, as most viewers expect and somehow, it gets away from that promise into an introspective view at a young man's soul. The film has a way of staying with us ev ...

 Horror moview #2:
The subject of children being terminally ill is difficult and saddening but 'The Cure' successfully portrays the idea that it doesn't have to be all doom and gloom and, if anything, children need to have hope and delight in their lives if they are to find peace before the end. It is also a film of r ...

 Horror moview #3:
I knew absolutely nothing about Chocolat before my viewing of it. I didn't know anything about the story, the cast, the director, or anything about the film's history. All I knew was it was a highly-acclaimed French film. Had I known more, I probably wouldn't have viewed the picture with an open min ...
